# Full Sherpa integration on a single systematic model

We revamped the entire main marginalization script to run with Sherpa. The script runs, but it is very slow to run on all 50 systems and there's also a lot of details we have to iron out. In this notebook, I will use a single systematic model to go through the entire thing and fix bugs.

In [ ]:
# Imports
import numpy as np
import os
import time
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import astropy.units as u
from astropy.constants import G

from sherpa.data import Data1D
from sherpa.plot import DataPlot
from sherpa.plot import ModelPlot
from sherpa.optmethods import LevMar, NelderMead
from sherpa.stats import Chi2
from sherpa.fit import Fit
from sherpa.estmethods import Confidence

os.chdir('../../../exotic-ism')
from config import CONFIG_INI
from limb_darkening import limb_dark_fit
import margmodule as marg

## Read inputs

### Configfile

We will first need to read a couple of things from the local configuration file. Make sure you know which one you're reading from (`config.ini` or `config_local.ini`) and that your parameters in there are correct.

In [ ]:
# Set up the paths
localDir = CONFIG_INI.get('data_paths', 'local_path')
outDir = CONFIG_INI.get('data_paths', 'output_path')
curr_model = CONFIG_INI.get('setup', 'data_set')
dataDir = os.path.join(CONFIG_INI.get('data_paths', 'input_path'), curr_model)

print('localDir: {}'.format(localDir))
print('outDir: {}'.format(outDir))
print('curr_model: {}'.format(curr_model))
print('dataDir: {}'.format(dataDir))

# What to call the run and whether to turn plotting on
run_name = CONFIG_INI.get('data_paths', 'run_name')
plotting = CONFIG_INI.getboolean('setup', 'plotting')

print('run_name: {}'.format(run_name))
print('plotting: {}'.format(plotting))

In [ ]:
# Parameters for smooth model
resolution = CONFIG_INI.getfloat('smooth_model', 'resolution')
half_range = CONFIG_INI.getfloat('smooth_model', 'half_range')

print('The smooth model will have {} data points.'.format(2 * half_range / resolution))

### Data

The x-array containsn the dates of observation, the y-array contains the flux, err is the error on the flux and sh contains systematic shifts.

In [ ]:
# Read the data
get_timeseries = CONFIG_INI.get(curr_model, 'lightcurve_file')
get_wvln = CONFIG_INI.get(curr_model, 'wvln_file')
x, y, err, sh = np.loadtxt(os.path.join(dataDir, get_timeseries), skiprows=7, unpack=True)
wavelength = np.loadtxt(os.path.join(dataDir, get_wvln), skiprows=3)

print('Data: {}'.format(curr_model))
print('x:\n{}'.format(x))
print('y:\n{}'.format(y))
print('err:\n{}'.format(err))
print('sh:\n{}'.format(sh))

In [ ]:
# Plot the raw data
plt.scatter(x, y, marker='o')
plt.title('Input data')

In [ ]:
# Plot data with errorbars
plt.errorbar(x, y, yerr=err)
plt.title('Input data with error bars')

The errorbars are so small, we can barely see them.

In [ ]:
# Plot the errors (because why not)
plt.plot(x, err)
plt.title('Input statistical errors')

Since the given errors are just photon noise, which is the square root of the signal, the errors also look like a transit curve.

In [ ]:
# Display the systematic shift
plt.scatter(x, sh, marker='o')
plt.hlines(0, np.amin(x), np.amax(x), linestyle='dashed', color='r')
plt.title('Systematic shift')

### Constants and planet starting parameters

In [ ]:
# READ THE CONSTANTS
HST_period = CONFIG_INI.getfloat('constants', 'HST_period') * u.d

# We want to keep the raw data as is, so we generate helper arrays that will get changed from model to model
img_date = x * u.d    # time array
img_flux = y    # flux array
flux0 = y[0]   # first flux data point
tzero = x[0] * u.d      # first time data point
nexposure = len(img_date)   # Total number of exposures in the observation

print('HST_period: {}'.format(HST_period))
print('flux0: {}'.format(flux0))
print('tzero: {}'.format(tzero))
print('nexposure: {}'.format(nexposure))

In [ ]:
# Planet starting parameters that are not directly read in the model
Per = CONFIG_INI.getfloat(curr_model, 'Per') * u.d    # period, converted to seconds in next line
Per = Per.to(u.s)

constant1 = ((G * np.square(Per)) / (4 * np.square(np.pi))) ** (1 / 3)
aor = CONFIG_INI.getfloat(curr_model, 'aor')    # this is unitless -> "distance of the planet from the star (meters)/stellar radius (meters)"
MsMpR = (aor / constant1) ** 3.     # density of the system in kg/m^3 "(Mass of star (kg) + Mass of planet (kg))/(Radius of star (m)^3)"

print('Per: {}'.format(Per))
print('aor: {}'.format(aor))
print('MsMpR: {}'.format(MsMpR))

## Limb darkening

Note how this is the only place in which we use the `wavelength` array.

In [ ]:
# Limb darkening
M_H = CONFIG_INI.getfloat(curr_model, 'metallicity')    # metallicity
Teff = CONFIG_INI.getfloat(curr_model, 'Teff')   # effective temperature
logg = CONFIG_INI.getfloat(curr_model, 'logg')   # log(g), gravitation

print('M_H: {}'.format(M_H))
print('Teff: {}'.format(Teff))
print('logg: {}'.format(logg))

# Define limb darkening directory, which is inside this package
limbDir = os.path.join('..', 'Limb-darkening')
ld_model = CONFIG_INI.get('setup', 'ld_model')
grat = CONFIG_INI.get('setup', 'grating')

print('limbDir: {}'.format(limbDir))
print('ld_model: {}'.format(ld_model))
print('grat: {}'.format(grat))

_uLD, c1, c2, c3, c4, _cp1, _cp2, _cp3, _cp4, _aLD, _bLD = limb_dark_fit(grat, wavelength, M_H, Teff,
                                                                         logg, limbDir, ld_model)

print('\nc1 - c4: {}, {}, {}, {}'.format(c1, c2, c3, c4))

## Read grid of systematic models

The grid contains all systematic models that we want to fit and at the end run the marginalization over. In each systematic model, a different set of model parameters is thawed to be fit for or frozen to be kept fixed.

In [ ]:
# SELECT THE SYSTEMATIC GRID OF MODELS TO USE
# 1 in the grid means the parameter is fixed, 0 means it is free
# grid_selection: either one from 'fix_time', 'fit_time', 'fit_inclin', 'fit_msmpr' or 'fit_ecc'
grid_selection = CONFIG_INI.get('setup', 'grid_selection')
grid = marg.wfc3_systematic_model_grid_selection(grid_selection)
nsys, nparams = grid.shape

print('Grid selection: {}'.format(grid_selection))
print('nsys: {}'.format(nsys))
print('nparams: {}'.format(nparams))

## Set up Sherpa and book keeping

### Arrays for scatter and fit parameter results

In [ ]:
#  SET UP THE ARRAYS
# save arrays for the first step through to get the err inflation
w_scatter = np.zeros(nsys)
w_params = np.zeros((nsys, nparams))

print('w_scatter.shape: {}'.format(w_scatter.shape))
print('w_params.shape: {}'.format(w_params.shape))

### Sherpa data object

Feed our data that we read in into a Sherpa data object.

In [ ]:
# Set up the Sherpa data model
# Instantiate a data object
tdata = Data1D('Data', x, y, staterror=err)
print('Data object')
print(tdata)

In [ ]:
# Plot the data with Sherpa
dplot = DataPlot()
dplot.prepare(tdata)
dplot.plot()

Print the statistical errors of the model.

In [ ]:
print(tdata.staterror)

### Sherpa transit model object

Instantiate a Sherpa model object with our custon transit model.

In [ ]:
print(tzero)
print(MsMpR)
print(c1)
print(c2)
print(flux0)

In [ ]:
# Set up the Sherpa transit model
tmodel = marg.Transit(tzero, MsMpR, c1, c2, c3, c4, flux0, name="TransitModel", sh=sh)
print('Starting parameters for the transit model:\n')
print(tmodel)

In [ ]:
# Plot the model
mplot = ModelPlot()
mplot.prepare(tdata, tmodel)
mplot.plot()

### Statistics and optimizer

We will use a chi squared statistic with a Levenberg-Marquardt optimizer. The optimizer has an attribute called `epsfcn` that needs to be set either to `FLT_EPSILON` or `DBL_EPSILON`, depending on how the model was written. Since our model is in double precision, this also needs to be machine precision.

In [ ]:
# Set up statistics and optimizer
stat = Chi2()
opt = LevMar()
opt.config['epsfcn'] = np.finfo(float).eps   # adjusting epsfcn to double precision

In [ ]:
# Print details about the statistics.
print(stat)

In [ ]:
# Print details about the optimizer
print(opt)

### Sherpa fit object from data and model objects

We instantiate a Sherpa fit object that contains a reference to a data object and a model object. If we change either of them, the change gets reflected in the fit object.

In [ ]:
# Set up the fit object
tfit = Fit(tdata, tmodel, stat=stat, method=opt)
tfit.estmethod = Confidence()
print('Fit information:')
print(tfit)

## First fit

We will work on one systematic model only, you can pick one by setting `i`.

In [ ]:
i = 0
sys = grid[i]

print('System {}: {}'.format(i+1, sys))

### Set up systematics

We will now thaw the parameters that have a `0` in the systematic grid and freeze the parameters that have a `1` in the systematic grid.

In [ ]:
# Set up systematics for current run
print('sys: {}'.format(sys))
for k, select in enumerate(sys):
    if select == 0:
        tmodel.pars[k].thaw()
    elif select == 1:
        tmodel.pars[k].freeze()

print(tmodel)

### Perform the fit

In [ ]:
print('\nSTART 1st FIT')
tres = tfit.fit()  # do the fit
if not tres.succeeded:
    print(tres.message)
print('\n1st ROUND OF SHERPA FIT IS DONE\n')

print('Fit result:')
print(tres)
print('\nFormatted result:')
print(tres.format())

It seems like `tres` already has errors for the thawed and fitted parameters (see formatted result), but are those correct and can I access those? If yes, I wouldn't have to do the error calculation further down.

The full API for the results object can be found here:  
https://sherpa.readthedocs.io/en/latest/fit/api/sherpa.fit.FitResults.html

In [ ]:
# Save results of fit
w_params[i, :] = [par.val for par in tmodel.pars]
    
print('w_params[{}]:'.format(i))
print(w_params[i])

In [ ]:
print('Parameters of transit model after first fit:\n')
print(tmodel)

### Calculate errors

We get our errors directly from the Hessian.

In [ ]:
calc_errors = np.sqrt(tres.extra_output['covar'].diagonal())

rl_err = calc_errors[0]
ph_err = calc_errors[1]

We can directly access the parameters values from the model.

In [ ]:
print('rl:\n{}'.format(tmodel.rl))
print('\nValue of rl: {}'.format(tmodel.rl.val))

In [ ]:
print('\nTRANSIT DEPTH rl in model {} of {} = {} +/- {}, centered at {}'.format(i+1, nsys, tmodel.rl.val, rl_err, tmodel.epoch.val))

### Recalculate phase

Since the phase array depends on the epoch, and the epoch gets changed during the fit, we need to recalculate the phase array after each fit.

In [ ]:
# Re-Calculate each of the arrays dependent on the output parameters
HSTphase = marg.phase_calc(img_date, tmodel.tzero.val*u.d, HST_period)
phase = marg.phase_calc(img_date, tmodel.epoch.val*u.d, tmodel.period.val*u.d)

plt.plot(phase, y)

### Calculate transit model from fit data

We can actually draw this from the Sherpa object directly. Not so the smooth model though that we'll make after the second fit.

In [ ]:
# TRANSIT MODEL fit to the data
b0 = marg.impact_param((tmodel.period.val*u.d).to(u.s), tmodel.msmpr.val, phase, tmodel.inclin.val*u.rad)
mulimb01, _mulimbf1 = marg.occultnl(tmodel.rl.val, tmodel.c1.val, tmodel.c2.val, tmodel.c3.val, tmodel.c4.val, b0)

print('Impact parameter: {}'.format(b0))
print('mulimb01: {}'.format(mulimb01))

In [ ]:
print('phase: {}'.format(phase))
print('HSTphase: {}'.format(HSTphase))
print('sh: {}'.format(sh))

In [ ]:
print('tmodel.m_fac.val: {}'.format(tmodel.m_fac.val))
print('tmodel.hstp1.val: {}'.format(tmodel.hstp1.val))
print('tmodel.xshift1.val: {}'.format(tmodel.xshift1.val))

In [ ]:
systematic_model = marg.sys_model(phase, HSTphase, sh, tmodel.m_fac.val, tmodel.hstp1.val, tmodel.hstp2.val,
                                  tmodel.hstp3.val, tmodel.hstp4.val, tmodel.xshift1.val, tmodel.xshift2.val,
                                  tmodel.xshift3.val, tmodel.xshift4.val)
print('systematic_model: {}'.format(systematic_model))

### Final form of model fit

We're combining the transit model, the systematic model and normalize it by the input flux. No need for this in the future, see above.

In [ ]:
# Calculate final form of the model fit
print('tmodel.flux0.val: {}'.format(tmodel.flux0.val))
w_model = mulimb01 * tmodel.flux0.val * systematic_model

print('w_model:\n{}'.format(w_model))
plt.plot(phase, w_model)
plt.title('w_model')

In [ ]:
# Calculate the residuals - data minus model (and normalized)
w_residuals = (img_flux - w_model) / tmodel.flux0.val

print('w_residuals:\n{}'.format(w_residuals))
plt.scatter(phase, w_residuals, marker='o')
plt.hlines(0, np.amin(phase)-0.01, np.amax(phase)+0.01, linestyle='dashed', color='r')
plt.title('w_residuals')

In [ ]:
corrected_data = img_flux / (tmodel.flux0.val * systematic_model)

print('corrected_data: {}'.format(corrected_data))
plt.plot(phase, corrected_data)
plt.title('corrected_data')

In [ ]:
w_scatter[i] = np.std(w_residuals)
print('\nScatter on the residuals = {}'.format(w_scatter[i])) 

### Reset the model

In [ ]:
print('Before resetting:')
print(tmodel)

In [ ]:
tmodel.reset()
print('After resetting:')
print(tmodel)

Note on resetting the model (from: https://sherpa.readthedocs.io/en/latest/models/index.html#resetting-parameter-values):

The `reset()` method of a parameter will change the parameter settings (which includes the status of the thawed flag and allowed ranges, as well as the value) to the values they had the last time the parameter was explicitly set. That is, it does not restore the initial values used when the model was created, but the last values the user set.

The model class has its own `reset()` method which calls reset on the thawed parameters. This can be used to change the starting point of a fit to see how robust the optimiser is by:

- explicitly setting parameter values (or using the default values)
- fit the data
- call reset
- change one or more parameters
- refit

## Second fit

Each systematic model will now be re-fit with the previously determined parameters serving as the new starting points. What we're doing is to replace the input data errors with the errors we calculated in the first fit, to make them more realistic. The uncertainty we get from the data is pure photon noise, which is not entirely realistic as there will be other noise source influencing the data. The first fit the additional noise sources. When we rescale the uncertainties with a first fit, we get a reduced chi squared of one, and the uncertainties will be slightly larger than the photon onise, giving us more conservative (and realistic) errors for the second fit, so that we can trust the parameters from the second fit.

In [ ]:
# Initializing arrays for each systematic model, which we will save once we got through all systems with two fits.
sys_stats = np.zeros((nsys, 5))                 # stats

sys_date = np.zeros((nsys, nexposure))          # img_date
sys_phase = np.zeros((nsys, nexposure))         # phase
sys_rawflux = np.zeros((nsys, nexposure))       # raw lightcurve flux
sys_rawflux_err = np.zeros((nsys, nexposure))   # raw lightcurve flux error
sys_flux = np.zeros((nsys, nexposure))          # corrected lightcurve flux
sys_flux_err = np.zeros((nsys, nexposure))      # corrected lightcurve flux error
sys_residuals = np.zeros((nsys, nexposure))     # residuals
sys_systematic_model = np.zeros((nsys, nexposure))  # systematic model

sys_model = np.zeros((nsys, int(2*half_range/resolution)))             # smooth model
sys_model_phase = np.zeros((nsys, int(2*half_range/resolution)))        # smooth phase

sys_params = np.zeros((nsys, nparams))          # parameters
sys_params_err = np.zeros((nsys, nparams))      # parameter errors

sys_depth = np.zeros(nsys)                      # depth
sys_depth_err = np.zeros(nsys)                  # depth error
sys_epoch = np.zeros(nsys)                      # transit time
sys_epoch_err = np.zeros(nsys)                  # transit time error
sys_evidenceAIC = np.zeros(nsys)                # evidence AIC
sys_evidenceBIC = np.zeros(nsys)                # evidence BIC

In [ ]:
# Still working only on one systematic model
print('i = {}'.format(i))
print('sys = {}'.format(sys))

### Rescale errors

The errors at this point got rescaled to unity chi squared in the previous fit and were not reset by `model.reset()`, so we ditched the part of the code that rescales the errors manually.

In [ ]:
# Check what the errors in the data object look like now.
print('New staterror data object')
print(tdata.staterror)

### Set up systematics

In [ ]:
# Set up systematics for current run
print('sys: {}'.format(sys))
for k, select in enumerate(sys):
    if select == 0:
        tmodel.pars[k].thaw()
    elif select == 1:
        tmodel.pars[k].freeze()

print(tmodel)

### Perform second fit

In [ ]:
print('\nSTART 2nd FIT\n')
tres = tfit.fit()  # do the fit
if not tres.succeeded:
    print(tres.message)
print('2nd ROUND OF SHERPA FIT IS DONE\n')

print('Fit result:')
print(tres)
print('\nFormatted result:')
print(tres.format())

### Calculate errors

Copy-pasted from the script since it was set up there.

In [ ]:
# Getting errors directly from the covariance matrix in the fit, rl is always thawed.
calc_errors = np.sqrt(tres.extra_output['covar'].diagonal())
rl_err = calc_errors[0]

# These are the only errors we might need, depending on "grid_selection"
epoch_err = None
incl_err = None
msmpr_err = None
ecc_err = None

# Read errors from Hessian depending on which parameters actually got fit
if grid_selection == 'fix_time':
    pass
elif grid_selection == 'fit_time':
    epoch_err = calc_errors[2]
elif grid_selection == 'fit_inclin':
    incl_err = calc_errors[2]
elif grid_selection == 'fit_msmpr':
    msmpr_err = calc_errors[2]
elif grid_selection == 'fit_ecc':
    ecc_err = calc_errors[2]
elif grid_selection == 'fit_all':
    epoch_err = calc_errors[2]
    incl_err = calc_errors[3]
    msmpr_err = calc_errors[4]

In [ ]:
print('\nTRANSIT DEPTH rl in model {} of {} = {} +/- {}, centered at {}'.format(i+1, nsys, tmodel.rl.val, rl_err, tmodel.epoch.val))

### Stats from fit

In [ ]:
# Count free parameters by figuring out how many zeros we have in the current systematics
nfree = np.count_nonzero(sys==0)
print('nfree: {}'.format(nfree))
print(sys)

The statistics can be taken from the fit result `tres`. you can check its full API here:  
https://sherpa.readthedocs.io/en/latest/fit/api/sherpa.fit.FitResults.html

In [ ]:
# From the fit define the DOF, BIC, AIC & CHI
CHI = tres.statval  # chi squared of resulting fit
BIC = CHI + nfree * np.log(len(img_date))
AIC = CHI + nfree
DOF = tres.dof

print('CHI: {}'.format(CHI))
print('BIC: {}'.format(BIC))
print('AIC: {}'.format(AIC))
print('DOF: {}'.format(DOF))

In [ ]:
# EVIDENCE BASED on the AIC and BIC
Npoint = len(img_date)
sigma_points = np.median(err)

evidence_BIC = - Npoint * np.log(sigma_points) - 0.5 * Npoint * np.log(2 * np.pi) - 0.5 * BIC
evidence_AIC = - Npoint * np.log(sigma_points) - 0.5 * Npoint * np.log(2 * np.pi) - 0.5 * AIC

print('Npoint: {}'.format(Npoint))
print('sigma_points: {}'.format(sigma_points))
print('evidence_BIC: {}'.format(evidence_BIC))
print('evidence_AIC: {}'.format(evidence_AIC))

In [ ]:
print('tmodel.period.val*u.d: {}'.format(tmodel.period.val*u.d))
print('tmodel.epoch.val*u.d: {}'.format(tmodel.epoch.val*u.d))
print('tmodel.tzero.val*u.d: {}'.format(tmodel.tzero.val*u.d))

# Recalculate a/R* (actually the constant for it) based on the new MsMpR value which may have been fit in the routine.
constant1 = (G * np.square((tmodel.period.val*u.d).to(u.s)) / (4 * np.pi * np.pi)) ** (1 / 3.)

# OUTPUTS
# Re-Calculate each of the arrays dependent on the output parameters for the epoch
phase = marg.phase_calc(img_date, tmodel.epoch.val*u.d, tmodel.period.val*u.d)
HSTphase = marg.phase_calc(img_date, tmodel.tzero.val*u.d, HST_period)

### Recalculate transit model from fit data

In [ ]:
# TRANSIT MODEL fit to the data
b0 = marg.impact_param((tmodel.period.val*u.d).to(u.s), tmodel.msmpr.val, phase, tmodel.inclin.val*u.rad)
mulimb01, _mulimbf1 = marg.occultnl(tmodel.rl.val, tmodel.c1.val, tmodel.c2.val, tmodel.c3.val, tmodel.c4.val, b0)

print('b0: {}'.format(b0))
print('mulimb01: {}'.format(mulimb01))

In [ ]:
# SMOOTH TRANSIT MODEL across all phase
x2 = np.arange(-half_range, half_range, resolution)   # this is the x-array for the smooth model
b0 = marg.impact_param((tmodel.period.val*u.d).to(u.s), tmodel.msmpr.val, x2, tmodel.inclin.val*u.rad)

print('x2:  {}'.format(x2))
print('b0: {}'.format(b0))

In [ ]:
mulimb02, _mulimbf2 = marg.occultnl(tmodel.rl.val, tmodel.c1.val, tmodel.c2.val, tmodel.c3.val, tmodel.c4.val, b0)

systematic_model = marg.sys_model(phase, HSTphase, sh, tmodel.m_fac.val, tmodel.hstp1.val, tmodel.hstp2.val,
                                  tmodel.hstp3.val, tmodel.hstp4.val, tmodel.xshift1.val, tmodel.xshift2.val,
                                  tmodel.xshift3.val, tmodel.xshift4.val)

print('mulimb02: {}'.format(mulimb02))
print('systematic_model: {}'.format(systematic_model))

In [ ]:
fit_model = mulimb01 * tmodel.flux0.val * systematic_model

print('fit_model: {}'.format(fit_model))
plt.plot(phase, fit_model)
plt.title('fit_model')

In [ ]:
# Can I access fit_model directly from the Sherpa model?
# v1: tmodel(tdata.x)
# v2: tdata.eval_model(tmodel)

In [ ]:
residuals = (img_flux - fit_model) / tmodel.flux0.val

print('residuals: {}'.format(residuals))
plt.scatter(phase, residuals)
plt.hlines(0, np.amin(phase)-0.01, np.amax(phase)+0.01, linestyle='dashed', color='r')
plt.title('residuals')

In [ ]:
resid_scatter = np.std(w_residuals)
print('resid_scatter: {}'.format(resid_scatter))

In [ ]:
fit_data = img_flux / (tmodel.flux0.val * systematic_model)

print('fit_data: {}'.format(fit_data))
plt.scatter(phase, fit_data)
plt.title('fit_data')

In [ ]:
plt.figure(figsize=(18, 12))
plt.clf()
plt.scatter(phase, img_flux, s=5, label='img_flux vs phase')
plt.plot(x2, mulimb02, 'k', label='mulimb02 vs x2')
plt.errorbar(phase, fit_data, yerr=err, fmt='m.', label='fit_data vs phase')
plt.xlim(-0.03, 0.03)
plt.title('Model ' + str(i+1) + '/' + str(nsys))
plt.xlabel('Planet Phase')
plt.ylabel('Data')
plt.legend()
plt.draw()
plt.pause(0.05)

In [ ]:
# Fill info into arrays to save to file once we iterated through all systems with both fittings.
sys_stats[i, :] = [AIC, BIC, DOF, CHI, resid_scatter]   # stats  - just saving

sys_date[i, :] = img_date                               # input time data (x, date)  - reused but not really
sys_phase[i, :] = phase                                 # phase  - used for plotting
sys_rawflux[i, :] = img_flux                            # raw lightcurve flux  - just saving
sys_rawflux_err[i, :] = err                             # raw flux error  - just saving
sys_flux[i, :] = fit_data                               # corrected lightcurve flux
sys_flux_err[i, :] = err                                # corrected flux error  - used for plotting
sys_residuals[i, :] = residuals                         # residuals   - REUSED! also for plotting
sys_systematic_model[i, :] = systematic_model           # systematic model  - just saving

sys_model[i, :] = mulimb02                              # smooth model  - used for plotting
sys_model_phase[i, :] = x2                              # smooth phase  - used for plotting

sys_params[i, :] = [par.val for par in tmodel.pars]     # parameters  - REUSED!
if not tmodel.inclin.frozen:                            # errors, depending on whether parameter is frozen or thawed
    sys_params_err[:, 3] = incl_err
if not tmodel.msmpr.frozen:
    sys_params_err[:, 4] = msmpr_err

sys_depth[i] = tmodel.rl.val                            # depth  - REUSED!
sys_depth_err[i] = rl_err                               # depth error  - REUSED!
sys_epoch[i] = tmodel.epoch.val                         # transit time  - REUSED!
if not tmodel.epoch.frozen:
    sys_epoch_err[i] = epoch_err                            # transit time error  - REUSED!
sys_evidenceAIC[i] = evidence_AIC                       # evidence AIC  - REUSED!
sys_evidenceBIC[i] = evidence_BIC                       # evidence BIC  - REUSED!

### Reset model

In [ ]:
print(tmodel)

In [ ]:
# Reset the model parameters to the input parameters
tmodel.reset()